In [19]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from langchain_core.output_parsers import StrOutputParser
from langchain.llms.vllm import VLLM
import evaluate
import json
from langchain_core.prompts import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# Define system prompt and human message

model_paths = ["/data/models/huggingface/meta-llama/Llama-2-7b-chat-hf"]
system_template = "You are a helpful assistant."
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('Paragraph: {question}')
]
prompt = ChatPromptTemplate.from_messages(messages)
output_parser = StrOutputParser()

In [3]:
# Now Loading LLama
sampling_params = SamplingParams(temperature=0, top_p=0.9, top_k=10, max_tokens=4096)
model = VLLM(model=model_paths[0], gpu_memory_utilization = 0.95, tensor_parallel_size=4, sampling_params = sampling_params)
chain = prompt | model

2024-04-20 21:11:24,221	INFO worker.py:1724 -- Started a local Ray instance.


INFO 04-20 21:11:27 llm_engine.py:72] Initializing an LLM engine with config: model='/data/models/huggingface/meta-llama/Llama-2-7b-chat-hf', tokenizer='/data/models/huggingface/meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, seed=0)
INFO 04-20 21:11:38 custom_all_reduce.py:125] NVLink detection failed with message "Not Supported". This is normal if your machine has no NVLink equipped
(RayWorkerVllm pid=3375504) INFO 04-20 21:11:38 custom_all_reduce.py:125] NVLink detection failed with message "Not Supported". This is normal if your machine has no NVLink equipped
INFO 04-20 21:11:43 llm_engine.py:322] # GPU blocks: 18067, # CPU blocks: 2048
INFO 04-20 21:11:45 model_runner.py:632] Capturing the model for CUDA graphs. This may 

[W CUDAGraph.cpp:145] Warning: Waiting for pending NCCL work to finish before starting graph capture. (function operator())
(RayWorkerVllm pid=3375504) [W CUDAGraph.cpp:145] Warning: Waiting for pending NCCL work to finish before starting graph capture. (function operator())


INFO 04-20 21:12:10 custom_all_reduce.py:199] Registering 715 cuda graph addresses
INFO 04-20 21:12:10 model_runner.py:698] Graph capturing finished in 25 secs.
(RayWorkerVllm pid=3375504) INFO 04-20 21:12:10 custom_all_reduce.py:199] Registering 715 cuda graph addresses
(RayWorkerVllm pid=3375504) INFO 04-20 21:12:10 model_runner.py:698] Graph capturing finished in 25 secs.
(RayWorkerVllm pid=3375822) INFO 04-20 21:11:45 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. [repeated 2x across cluster]
(RayWorkerVllm pid=3375822) INFO 04-20 21:11:45 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage. [repeated 2x across 

In [10]:
# Prepare Data

file_path ="alpaca_format.json"
# load processed marvel data set
with open(file_path, 'r') as file:
# Parse the JSON file and convert it into a Python dictionary
    data = json.load(file)


In [26]:
# Test on the first sample

q = data[0]['instruction']
pair = {"question":q}

answer = chain.invoke(pair)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.97it/s]


In [27]:
print(answer)



The new police chief will have to work hard to restore the public's trust in the department, which has been severely tested in recent months.


In [28]:
>>> rouge = evaluate.load('rouge')
>>> predictions = [answer]
>>> references = [data[0]['expected_output']]
>>> results = rouge.compute(predictions=predictions,
...                         references=references)
>>> print(results)

{'rouge1': 0.21621621621621623, 'rouge2': 0.05555555555555556, 'rougeL': 0.21621621621621623, 'rougeLsum': 0.21621621621621623}
